# Testing RAG and Retrieve in Amazon Bedrock

## Reason
This notebook demonstrates how to implement and test Retrieval-Augmented Generation (RAG) and retrieve functionality using Amazon Bedrock. It explores various approaches to enhance language models with external knowledge.

## Sections
- [Test a knowledge base](#test-a-knowledge-base)
- [Test a persistent conversation with RAG](#test-persistent-conversation-with-rag)

In [1]:
import boto3
import pprint

In [2]:
# import kb_id from my private file config.py
from config import *

<a id="test-a-knowledge-base"></a>
### Test a knowledge base
- Installing necessary libraries
- Setting up Bedrock client
- Configuring model parameters

In [3]:
kb_id = kb_id
pp = pprint.PrettyPrinter(indent=2)
region_name = 'us-east-1'
bedrock_agent_runtime_client = boto3.client('bedrock-agent-runtime', region_name = region_name)
model_id =  ["Claude 3 Sonnet", "anthropic.claude-3-sonnet-20240229-v1:0"]

In [4]:
def ask_bedrock_llm_with_knowledge_base(query: str, model_arn: str, kb_id: str, sessionId:str = None) -> str:
    response = "None"
    print(f"sessionId: {sessionId}")
    # If this is the first RAG action
    if not sessionId:
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={
                'text': query
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': model_arn
                }
            },
        )
    else:
        # If sessionId is present
        print(f"sessionId: {sessionId}")
        response = bedrock_agent_runtime_client.retrieve_and_generate(
            input={
                'text': query
            },
            retrieveAndGenerateConfiguration={
                'type': 'KNOWLEDGE_BASE',
                'knowledgeBaseConfiguration': {
                    'knowledgeBaseId': kb_id,
                    'modelArn': model_arn
                }
            },
            sessionId = sessionId
        )       

    return response

In [5]:
query = 'list 5 insignificant packages listted in the exported environments'


model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id[1]}'
response = ask_bedrock_llm_with_knowledge_base(query, model_arn, kb_id)
# !important: keep the same session Id for the follow up questions
sessionId = response['sessionId']
generated_text = response['output']['text']
# the list of citations from the knowledge base
citations = response["citations"]
contexts = []

# extra: print the citations from the knowledgebase
#for citation in citations:
#    retrievedReferences = citation["retrievedReferences"]
#    for reference in retrievedReferences:
#        contexts.append(reference["content"]["text"])
print(f"---------- Generated using {model_id[0]}:")
pp.pprint(generated_text)
print(f'---------- The citations for the response generated by {model_id[0]}:')
pp.pprint(contexts)
print()

sessionId: None
---------- Generated using Claude 3 Sonnet:
('Here are 5 relatively insignificant packages listed in the exported '
 'environments:\n'
 '\n'
 '1. xsel (a command-line program for manipulating the X Window selection)\n'
 '2. xorg-libxmu (X.Org X11 miscellaneous utility library)\n'
 '3. xorg-libxt (X.Org X11 toolkit intrinsics library)\n'
 '4. xorg-libice (X.Org Inter-Client Exchange library)\n'
 '5. xorg-libxau (X.Org X11 authorisation library)')
---------- The citations for the response generated by Claude 3 Sonnet:
[]



<a id="test-persistent-conversation-with-rag"></a>
### Test a persistent conversation with RAG
- Installing necessary libraries
- Setting up Bedrock client
- Configuring model parameters

In [6]:
# we will use the same code as before, but keeping the same sessionId
query2 = 'what did you told me in the last response?'


model_arn = f'arn:aws:bedrock:{region_name}::foundation-model/{model_id[1]}'
response = ask_bedrock_llm_with_knowledge_base(query2, model_arn, kb_id,sessionId=sessionId)
# !important: keep the same session Id for the follow up questions
sessionId = response['sessionId']
generated_text = response['output']['text']
citations = response["citations"]
contexts = []

# extra: print the citations from the knowledgebase
#for citation in citations:
#    retrievedReferences = citation["retrievedReferences"]
#    for reference in retrievedReferences:
#        contexts.append(reference["content"]["text"])
print(f"---------- Generated using {model_id[0]}:")
pp.pprint(generated_text)
print(f'---------- The citations for the response generated by {model_id[0]}:')
pp.pprint(contexts)
print()

sessionId: 5c1b8c71-4599-4017-9273-e12f79ad0f14
sessionId: 5c1b8c71-4599-4017-9273-e12f79ad0f14
---------- Generated using Claude 3 Sonnet:
('Based on the search results, here are 5 packages that seem relatively '
 'insignificant or small in scope that are listed in the exported '
 'environments:\n'
 '\n'
 '1. pluggy (version 1.5.0)\n'
 '2. ptyprocess (version 0.7.0)\n'
 '3. pure_eval (version 0.2.3)\n'
 '4. pygments (version 2.18.0)\n'
 '5. pymsgbox (version 1.0.9)')
---------- The citations for the response generated by Claude 3 Sonnet:
[]

